In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import re

BLOCK = 10 
PAGE_LOAD = 100 #This could be 100, 50 or 10

def extract_wallet_address(column):
    pattern = r"\((0x[0-9a-fA-F]+)\)"
    match = re.search(pattern, str(column))

    if match:
        wallet_address = match.group(1)
    else:
        pattern2 = r"data-clipboard-text=\"0x[0-9a-fA-F]+"
        match = re.search(pattern2, str(column))

        if match:
            wallet_address = match.group().replace('data-clipboard-text=\"', '')
        else:
            wallet_address = None
            
    return wallet_address

options = webdriver.FirefoxOptions()
#options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # no GUI#
#driver = webdriver.Chrome(options=options)
driver = webdriver.Firefox(options=options)

block_counter = 0
page_counter = 1
prev = []

hash_list = []
block_list = []
age_list = []
from_list = []
to_list = []
value_list = []
txn_fee_list = []
    
while(block_counter != BLOCK + 1):
    url = 'https://etherscan.io/txs?ps=' + str(PAGE_LOAD) +'&p=' + str(page_counter)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('tbody')
    rows = table.find_all('tr')[1:]  
   
    for row in rows[:PAGE_LOAD]:
        columns = row.find_all('td')    
        current_block = columns[3].text.strip()
        
        if(current_block not in prev):
            prev.append(current_block)
            block_counter += 1
        
        if(block_counter > BLOCK + 1):
            break;
        
        hash_list.append(columns[1].text.strip())
        block_list.append(current_block)
        age_list.append(columns[5].text.strip())
        from_list.append(extract_wallet_address(columns[7]))
        to_list.append(extract_wallet_address(columns[9]))
        value_list.append(columns[10].text.strip())
        txn_fee_list.append(columns[11].text.strip())
        
    page_counter += 1

df = pd.DataFrame({
    'Hash': hash_list,
    'Block': block_list,
    'Age': age_list,
    'From': from_list,
    'To': to_list,
    'Value': value_list,
    'Txn Fee': txn_fee_list
})

driver.quit()
df.to_csv('output.csv', index=False)

<b>Data Gathering Report</b>

By using BeautifulSoup and selenium libraries we gather transactions of 10 blocks.
There are 2 const variables called BLOCK and PAGE_LOAD that represent the number of blocks the user wants and how many transactions the user wants to gather in each scrabing.

There is 2 options for webdriver either FireFox or Google Chrome that is commented(In my device using Google Chrome gets privacy error but FireFox opens in a perfect way it might be different in your IP connection)

Let's get into algorithm the form of url link is https://etherscan.io/txs?ps=100&p=5 which the ps represents how many transactions are load in each url and the p represents the page number so till we gather 10 blocks transactions we're gathering all the transactions of page and then going to the next page.
Extracting the data of each transaction and putting them to their corresponding list. After finishing the gathering we puth all the dat to a panda dataframe and save it into output.csv

ChatGPT
Data Gathering Report

By utilizing the BeautifulSoup and Selenium libraries, we collected transactions from 10 blocks. There are two constant variables, BLOCK and PAGE_LOAD, which determine the number of blocks the user wishes to gather and how many transactions the user wants to scrape in each iteration.

There are two options for the webdriver: either Firefox or Google Chrome, which are commented out. (Note: On my device, using Google Chrome results in a privacy error, but Firefox opens perfectly. This may vary depending on your IP connection.)

Let's delve into the algorithm: the URL format is https://etherscan.io/txs?ps=100&p=5, where 'ps' represents the number of transactions loaded in each URL, and 'p' represents the page number. Therefore, to gather transactions from 10 blocks, we iterate through each page, extracting the data of each transaction and appending it to its corresponding list. Once gathering is complete, we compile all the data into a pandas dataframe and save it to output.csv.